**Downloading Dependencies**

In [ ]:
#@title Installing Dependencies
!gdown --id 19qX8boFv_3o01gsbs-YLTZKrpfTVBMDJ
!pip install pydub pysndfx pytube youtube-search-python  youtube_dl librosa demucs ruptures msaf scipy==1.2  essentia
!apt-get install sox
!pip install ffmpeg-python 
!pip install youtube_dl
!apt-get install libsox-fmt-all libsox-dev sox
print('Installation finished.')
# from IPython.display import clear_output
# clear_output()
!gdown --id 12N6OEV6GlBA7gAlQkYXRFgEUki_VcQFC
!unzip Justin-Bieber.zip
!rm -rf Justin-Bieber.zip

# **Downloading Videos**

In [ ]:
#@title Run to download videos
from __future__ import unicode_literals
import youtube_dl
from youtubesearchpython import VideosSearch
import threading
import time
from __future__ import unicode_literals
import youtube_dl
from youtubesearchpython import VideosSearch
!rm -rf /content/downloadedsongs
!mkdir downloadedsongs


def downloader(link):
    print("Wait for sometime, Downloading in background..")
    ydl_opts = {
        'outtmpl': 'downloadedsongs/%(title)s.%(ext)s',
        'format': 'bestaudio/best',
        # 'max_filesize':'90',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])


searchstring=input("Enter your song search term : ")
limit=int(input("Max number of videos :: "))
videosSearch = VideosSearch(searchstring, limit = limit)
links=[]
for i in range(limit):
    links.append(videosSearch.result()['result'][i]['link'])
    try:
        th = threading.Thread(target=downloader,args=(videosSearch.result()['result'][i]['link'],))
        th.start()
    except:
        print ("Error: unable to start thread")


# **Extract Segments**

In [ ]:
#@title Run for Video Segmentation
import msaf
import librosa
import numpy as np
import numpy as np
import librosa
import warnings
warnings.filterwarnings("ignore", category=Warning)
import essentia
import essentia.standard as es
import glob
from pysndfx import AudioEffectsChain
from pydub import AudioSegment
from scipy.io import wavfile
import librosa, numpy as np
import numpy as np
import soundfile as sf
import IPython
import matplotlib.pyplot as plt
import json


def get_label_groups(y, sections, labels):
    '''
    Parses labels and sections to generate a mapping of labels to actual sections and a measure of how percussive each section is
    
    Parameters
    ----------
    y: Audio signal
    sections: Array of tuples that describe the end and start times of each section
    labels: Array describing sections relations to one another. Signals that share similarities have the same label.
        
    Returns
    -------
    label_groups:
        A Dictionary from labels to their corresponding sections. 
        E.g. 
        If sections 0, 1 and 3 share similarities and sections 2, 4 share similarities
        {
            1: [(0, p), (1, p), (3, p)],
            2: [(2, p), (4, p)],
        }
        where each `p` is the percussive score of its corresponding section.
    '''

    label_groups = {}
    for i in range(len(labels)):
        if labels[i] not in label_groups:
            label_groups[labels[i]] = []
        percussiveness = librosa.effects.percussive(y[sections[i][0]:sections[i][1]])
        score = sum(abs(percussiveness))
        label_groups[labels[i]].append((i, score))

    return label_groups
    
def clean_labels(_labels):
    '''
    Makes label ids start at 0 and end at num_unique_sections 

    Parameters
    ----------
    _labels: old labels that need to be cleaned
        
    Returns
    -------
    labels:
        Cleaned labels
    '''

    labels = np.copy(_labels)
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    label_map = {}

    current_index = 0
    for i in range(len(labels)):
        if 'tro' in labels[i]:
            continue
        if labels[i] in label_map:
            labels[i] = alphabet[label_map[labels[i]]]
        else:
            label_map[labels[i]] = current_index
            labels[i] = alphabet[current_index]
            current_index += 1
    
    return labels

def clean_tempos(y, sr, sections):
    '''
    Generates and cleans the estimated tempos of each section by 
    checking for either accidental halving/doubling of tempos
    
    Parameters
    ----------
    y: audio signal
    sections: Array of tuples that describe the end and start times of each section
        
    Returns
    -------
    cleaned_tempos:
        An array of cleaned_tempos
    '''
    tempos = []
    for start, end in sections:
        tempos.append(librosa.beat.tempo(y[start:end], sr)[0])

    cleaned_tempos = [tempos[0]]
    for i in range(len(tempos)-1):
        ratio = tempos[i + 1] / tempos[i]
        if round(ratio) >= 2:
            cleaned_tempos.append(tempos[i+1] / round(ratio))
        elif round(1/ratio) >= 2:
            tempos[i] *= round(ratio)
            cleaned_tempos.append(tempos[i+1])
        else:
            cleaned_tempos.append(tempos[i+1])
        
    return cleaned_tempos

def get_beats_from_boundaries(beats, sections, sr):
    '''
    Converts the boundary values in seconds to beats by snapping the section boundaries to beat values
    
    Parameters
    ----------
    beats: estimated beat values in frames
    sections: Array of tuples that describe the end and start times of each section
    sr: sample rate
        
    Returns
    -------
    section_beats:
        An array of tuples that represent the start/end of each section in terms of beats
    '''

    section_beats = []
    beats = librosa.frames_to_time(beats, sr)
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx
    
    for start, end in sections:
        start_s = start/sr
        end_s = end/sr

        start_b = find_nearest(beats, start_s)
        end_b = find_nearest(beats, end_s)

        section_beats.append((int(start_b), int(end_b)))

    return section_beats


def get_clean_labels(y, sections, dirty_labels):
    '''
    Cleans dirty labels to produce more easily readable labels.
    
    Parameters
    ----------
    y: audio_signal
    sections: the sections list generated by `get_boundaries()`
    dirty_labels: the labels list generated by `get_labels()`
        
    Returns
    -------
    labels: cleaned label output.
        If the intro/outro sections are unique, they will be replaced by 'intro' and 'outro'
        A dirty_labels list that looks like [0, 1, 2, 0, 1] will be converted to ['A', 'B', 'C', 'A', 'B']
    '''
    label_groups = get_label_groups(y, sections, dirty_labels)

    if len(label_groups[dirty_labels[0]]) == 1:
        dirty_labels[0] = 'intro'
    if len(label_groups[dirty_labels[-1]]) == 1:
        dirty_labels[-1] = 'outro'

    labels = clean_labels(dirty_labels)

    return labels

def get_boundaries(audio_file: str, sample_rate: int, boundary_detection_id: str = "olda", label_detection_id: str = "scluster"):
    '''
    Parses labels and sections to generate a mapping of labels to actual sections and a measure of how percussive each section is
    
    Parameters
    ----------
    audio_file: filename of desired audio file
    sample_rate: sample_rate of audio file
    boundary_detection_id: 
        Algorithm for boundary detection. 
        Other algorithms are documented here: https://github.com/urinieto/msaf/blob/master/examples/Run%20MSAF.ipynb
    label_detection_id: 
        Algorithm for label_detection. 
        Other algorithms are documented here: https://github.com/urinieto/msaf/blob/master/examples/Run%20MSAF.ipynb
        
    Returns
    -------
    sections:
        A list of tuples designating the start/end of each section in samples
    dirty_labels:
        A list of integers corresponding to the label each section has been assigned 
        (e.g. a song structure of ABCAB could have a list of [0, 1, 2, 0, 1])
    '''

    bounds, labels = msaf.process(audio_file, boundaries_id=boundary_detection_id, labels_id=label_detection_id, out_sr=sample_rate)
    sections = np.array([( int(bounds[i] * sample_rate), int(bounds[i+1] * sample_rate) ) for i in range(len(bounds)-1)])
    boolarr = [True if end - start > sample_rate else False for start, end in sections]
    sections = sections[boolarr]
    labels = np.array(labels)[boolarr]
    dirty_labels = [int(i) for i in labels]

    return sections, dirty_labels

def read_audio(audio_file: str, sample_rate: int):
    '''
    Wrapper for librosa's load function to read in an audio signal
    
    Parameters
    ----------
    audio_file: filename of desired audio file
    sample_rate: sample_rate of audio file
        
    Returns
    -------
    y: audio signal
    sr: sample_rate
    '''

    y, sr = librosa.load(audio_file, sr = sample_rate)

    return y, sr

def get_section_beats(y, sr: int, sections):
    '''
    Gets the beat numbers of each section boundary based on estimated beat values
    
    Parameters
    ----------
    y: audio_signal
    sr: sample_rate
    sections: the sections list generated by `get_boundaries()`
        
    Returns
    -------
    section_beats: list of tuples corresponding to the section boundaries for each section in beats
    '''

    beats = librosa.beat.beat_track(y, sr)
    section_beats = get_beats_from_boundaries(beats[1], sections, sr)

    return section_beats

def get_tempos(y, sr: int, sections):
    '''
    Gets and cleans estimated tempo values for each section to analyse change in tempo
    
    Parameters
    ----------
    y: audio_signal
    sr: sample_rate
    sections: the sections list generated by `get_boundaries()`
        
    Returns
    -------
    tempos: list of cleaned tempos corresponding to each section
    '''

    tempos = clean_tempos(y, sr, sections)
    
    return tempos

def get_structure_array(sr: int, sections, section_beats, labels, tempos,global_tempo,key_chord):
    '''
    Formats basic structure information in an array format
    
    Parameters
    ----------
    sr: sample_rate
    sections: the sections list generated by `get_boundaries()`
    labels: the labels list generated by `get_boundaries()`
    section_beats: the section_beats list generated by `get_section_beats()`
    tempos: the tempos list generated by `get_tempos()`
        
    Returns
    -------
    output: A list where each value corresponds to a section, and each section contains the metadata:
        length_seconds: length of the section in seconds
        length_beats: length of the section in beats
        label: the label that this section was assigned
        tempo: the estimated tempo of the section
    '''

    output = []
    for section_no in range(len(sections)):
        output.append({
            'length_seconds': (sections[section_no][1] - sections[section_no][0])/sr,
            'length_beats': section_beats[section_no][1] - section_beats[section_no][0],
            'label' : labels[section_no],
            'tempo' : tempos[section_no],
            'start_time': sections[section_no][0]/sr,
            'end_time':sections[section_no][1]/sr,
            'beat_start':section_beats[section_no][0],
            'beat_end':section_beats[section_no][1],
            'global_tempo':global_tempo,
            'key_chord':key_chord

        })
    
    return output

def process(audio_file: str, sample_rate: int, boundary_detection_id: str = "olda", label_detection_id: str = "scluster", global_tempo=None,key_chord=None):
    '''
    Formats basic structure information in regular 
    
    Parameters
    ----------
    audio_file: filename of the audio file
    sample_rate: sample rate of the audio file
        
    Returns
    -------
    output: A list where each value corresponds to a section, and each section contains the metadata:
        length_seconds: length of the section in seconds
        length_beats: length of the section in beats
        label: the label that this section was assigned
        tempo: the estimated tempo of the section
    '''

    y, sr = read_audio(audio_file, sample_rate)
    sections, labels = get_boundaries(audio_file, sr, boundary_detection_id, label_detection_id)

    section_beats = get_section_beats(y, sr, sections)
    tempos = get_tempos(y, sr, sections)
    labels = get_clean_labels(y, sections, labels)

    return get_structure_array(sr, sections, section_beats, labels, tempos,global_tempo,key_chord)


# ---------------------------------------------

import os
from pydub.utils import mediainfo
import essentia.standard
from pydub import AudioSegment
import pandas as pd
import glob
import threading

def getSegments(i):
    try:
        y,sr=librosa.load(i)
        meanBPM=100
        tempo,beats=librosa.beat.beat_track(y,sr)

        if(tempo>140):
            tempo/=2
        if(tempo<65):
            tempo*=2

        song_faster = librosa.effects.time_stretch(y,rate=meanBPM/tempo)
        new=i.replace('.mp3','_100_.wav')

        sf.write(new, song_faster, sr, format='wav', subtype='PCM_24')


        features, features_frames = es.MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                                    rhythmStats=['mean', 'stdev'],
                                                    tonalStats=['mean', 'stdev'])(new)

        key=str(features['tonal.key_edma.key'])+":"+str(features['tonal.key_edma.scale'])

        out = process(audio_file=new,sample_rate=sr,global_tempo= float(features['rhythm.bpm']),key_chord=key)
        outname = i[i.rindex('/')+1:-3]+"csv"
        data= pd.DataFrame(out)
        # data = data[~data['length_beats'].isin([1,2,3,5,6]) or data['length_beats']>7 ]
        data.to_csv(outname)
    except Exception as e:
        print(e)
        os.remove(i)
        os.remove(new)

In [ ]:
#@title EXTRACTING SEGMENTS
import os
import glob
files=(glob.glob("/content/Justin Bieber - Justice (24Naijamuzic.com)/*.mp3"))
print("Wait till you See the CSV files ! ")
for i in files:
    print(i)
    try:
        getSegments(i)
    except Exception as e:
        print(e)

# files=(glob.glob("/content/King of Love Album/*.mp3"))
# for i in files:
#     th = threading.Thread(target=getSegments,args=(i,))
#     th.start()
# print("Wait till you See the CSV files ! ")

In [ ]:
#@title GETING THE BASE TRACK READY
drum=AudioSegment.from_file('drumloop.wav')
while drum.duration_seconds<=240:
    drum+=drum
drum.export("drum.wav", format="wav") 
y,sr=librosa.load('/content/drum.wav')
tempo,beat=librosa.beat.beat_track(y,units='time')
tempo

In [ ]:
#@title Generating Segments
import random
from IPython.display import Audio, display
def generate_random_number(max_num):
    random_number = random.randint(1,max_num-1)
    return random_number

def generate_random_number_list(listSize,max_num):
    random_number_list = []
    for i in range(listSize):
        random_number = generate_random_number(max_num)
        random_number_list.append(random_number)
    return random_number_list


files=(glob.glob("/content/Justin Bieber - Justice (24Naijamuzic.com)/*.mp3"))
fileNumber= generate_random_number_list(10,len(files))
newsong= AudioSegment.silent(duration=beat[0])
for i in range(len(fileNumber)):
    name=files[fileNumber[i]]
    csv=name[name.rindex('/')+1:-3]+"csv"
    song = AudioSegment.from_file(name.replace('.mp3','_100_.wav'))
    data=pd.read_csv(csv)
    songListLength=data.shape[0]
    seq= (generate_random_number(songListLength))
    print("File Number : ",fileNumber[i],"  Segment Number : ",seq)
    newsong+=AudioSegment.silent(duration=round((beat[beat>newsong.duration_seconds ].min()-newsong.duration_seconds)*1000))
    newsong+=song[data['start_time'].iloc[seq]*1000:data['end_time'].iloc[seq]*1000]
newsong.export("mashup.wav", format="wav")
display(Audio('/content/mashup.wav', rate=sr))

In [ ]:
#@title Generating Final MIX
!demucs --two-stems=drums 'mashup.wav'
no_drum=AudioSegment.from_file('/content/separated/mdx_extra_q/mashup/no_drums.wav','wav')
drum=AudioSegment.from_file('drumloop.wav')
while drum.duration_seconds<=no_drum.duration_seconds:
    drum+=drum
mashup=no_drum.overlay(drum)
mashup.export("mashupMusic.wav", format="wav")
display(Audio('/content/mashupMusic.wav', rate=sr))

In [ ]:
from google.colab import files
files.download('mashupMusic.wav') 